# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [74]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [75]:
vacation_df = pd.read_csv('../WeatherPy/WeatherPy.csv')
vacation_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fortuna,40.5982,-124.1573,18.89,72.0,1.0,6.17,US,1.619735e+09
1,1,geraldton,-28.7667,114.6000,18.00,82.0,90.0,5.66,AU,1.619735e+09
2,2,puerto ayora,-0.7393,-90.3518,27.00,78.0,20.0,7.20,EC,1.619735e+09
3,3,saint-philippe,-21.3585,55.7679,25.00,83.0,75.0,7.20,RE,1.619735e+09
4,4,castro,-24.7911,-50.0119,12.65,82.0,9.0,2.81,BR,1.619735e+09
...,...,...,...,...,...,...,...,...,...,...
634,634,jawhar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
635,635,linares,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
636,636,balclutha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
637,637,georgiyevka,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
vacation_df.dropna()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fortuna,40.5982,-124.1573,18.89,72.0,1.0,6.17,US,1.619735e+09
1,1,geraldton,-28.7667,114.6000,18.00,82.0,90.0,5.66,AU,1.619735e+09
2,2,puerto ayora,-0.7393,-90.3518,27.00,78.0,20.0,7.20,EC,1.619735e+09
3,3,saint-philippe,-21.3585,55.7679,25.00,83.0,75.0,7.20,RE,1.619735e+09
4,4,castro,-24.7911,-50.0119,12.65,82.0,9.0,2.81,BR,1.619735e+09
...,...,...,...,...,...,...,...,...,...,...
577,577,gizo,2.7809,45.5005,26.05,79.0,98.0,2.92,SO,1.619735e+09
578,578,gat,-35.8500,-71.6000,18.69,42.0,100.0,1.38,CL,1.619735e+09
579,579,capreol,-46.2339,169.7500,12.78,70.0,100.0,1.79,NZ,1.619735e+09
580,580,wewak,43.0369,74.7114,18.00,48.0,0.0,4.00,KZ,1.619735e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [77]:
gmaps.configure(api_key=g_key)
locations = vacation_df[["Lat","Lng"]]
humidity = vacation_df["Humidity"].astype(float)

In [78]:
locations = locations.dropna()
humidity = humidity.dropna()

In [79]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [102]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

ideal_weather = vacation_df.loc[(vacation_df["Wind Speed"] < 10) &
                                (vacation_df["Cloudiness"] == 0) &
                                (vacation_df["Max Temp"] >= 20) & 
                                (vacation_df["Max Temp"] <= 40)]

ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,33,pangnirtung,-12.0432,-77.0282,21.11,77.0,0.0,4.63,PE,1.619735e+09
41,41,belle fourche,-8.2578,-49.2647,24.00,88.0,0.0,2.57,BR,1.619735e+09
79,79,qaanaaq,-23.8650,35.3833,23.20,77.0,0.0,3.97,MZ,1.619735e+09
136,136,saleaula,41.1864,16.6674,20.56,48.0,0.0,3.09,IT,1.619735e+09
144,144,yanan,-38.0023,-57.5575,20.00,72.0,0.0,2.57,AR,1.619735e+09
150,150,esperance,12.1500,-68.2667,27.00,78.0,0.0,3.60,BQ,1.619735e+09
197,197,brae,27.9769,-114.0611,24.13,40.0,0.0,6.10,MX,1.619735e+09
216,216,kuche,33.3406,44.4009,26.00,27.0,0.0,3.09,IQ,1.619735e+09
243,243,te anau,41.7597,86.1469,20.03,17.0,0.0,2.24,CN,1.619735e+09
250,250,horsham,40.4272,71.7189,20.00,67.0,0.0,1.03,UZ,1.619735e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
